### Connect to Oracle 23ai

In [78]:
import oracledb
import sys

password = "<CUSTOMER_PASSWORD>"
conn=oracledb.connect(
     config_dir="LOCATION_TO_WALLET_DIR",
     user="USER_NAME",
     password=password,
     dsn="DSN_REQUIRED",
     wallet_location="LOCATION_TO_WALLET_DIR",
     wallet_password=password)

In [2]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import oraclevs
from langchain_community.vectorstores.oraclevs import OracleVS
from langchain_community.vectorstores.utils import DistanceStrategy
from langchain_core.documents import Document


### Upload the Embeddings to Oracle 23AI

In [1]:
from langchain_community.embeddings.oracleai import OracleEmbeddings

# please update with your related information
# make sure that you have onnx file in the system
onnx_dir = "ONNX_DIRECTORY"
onnx_file = "ONNX_FILE_NAME"
model_name = "MODEL_NAME" 

try:
    OracleEmbeddings.load_onnx_model(conn, onnx_dir, onnx_file, model_name)
    print("ONNX model loaded.")
except Exception as e:
    print("ONNX model loading failed!")
    sys.exit(1)

## Connect to AQUA LLM

In [95]:
from langchain_community.llms import OCIModelDeploymentVLLM
import ads

ads.set_auth("api_key")

oci_md = OCIModelDeploymentVLLM(
    endpoint="MODEL_DEPLOYMENT_ENDPOINT",
    model="/opt/ds/model/deployed_model"
)

In [96]:
oci_md.generate(prompts=["What is the National Capital of India?"])

LLMResult(generations=[[Generation(text='\n\nThe national capital of India is New Delhi. New Delhi is the capital city of India and is located in the north of the country. It is home to many important government buildings, institutions, and landmarks, including the Rashtrapati Bhavan, the Parliament House, and the India Gate. New Delhi is also the seat of the executive, legislative, and judicial branches of the Indian government.')]], llm_output=None, run=[RunInfo(run_id=UUID('b7c4370b-21a8-46c3-87e4-d06e82530914'))])

### Load Embeddings

In [8]:
embeddings = HuggingFaceEmbeddings(model_name="MODEL_NAME")

### Get Sample Documents to Upload to 23ai

In [11]:
!pip install xmltodict


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [73]:
from langchain_community.document_loaders import PubMedLoader
loader = PubMedLoader("diabetes", load_max_docs=200)

In [74]:
docs = loader.load()
len(docs)
print(docs[0].metadata)
print(docs[0].page_content)

{'uid': '38808578', 'Title': {'sub': 'Ca', '#text': 'mitoBK is functionally expressed in murine and human breast cancer cells and potentially contributes to metabolic reprogramming.'}, 'Published': '2024-05-29', 'Copyright Information': '© 2023, Bischof et al.'}
['+', '2+', '+']
['Ca', 'Ca', 'Ca', 'Ca', 'Ca', 'Ca', 'Ca', 'Ca']
Alterations in the function of K channels such as the voltage- and Ca-activated K channel of large conductance (BK) reportedly promote breast cancer (BC) development and progression. Underlying molecular mechanisms remain, however, elusive. Here, we provide electrophysiological evidence for a BK splice variant localized to the inner mitochondrial membrane of murine and human BC cells (mitoBK). Through a combination of genetic knockdown and knockout along with a cell permeable BK channel blocker, we show that mitoBK modulates overall cellular and mitochondrial energy production, and mediates the metabolic rewiring referred to as the 'Warburg effect', thereby promo

### Split Documents

In [2]:
from langchain_community.document_loaders.oracleai import OracleTextSplitter
from langchain_core.documents import Document

# split by default parameters
splitter_params = {"normalize": "all"}

""" get the splitter instance """
splitter = OracleTextSplitter(conn=conn, params=splitter_params)
count = 0
list_chunks = []
for doc in docs:
    print(count)
    count += 1
    chunks = splitter.split_text(doc.page_content)
    list_chunks.extend(chunks)

""" verify """
print(f"Number of Chunks: {len(list_chunks)}")

Number of Chunks: 855


### Upload docs to Vector Store 23ai

In [80]:
vector_store_dot = OracleVS(
    client=conn,
    embedding_function=embeddings,
    table_name="Documents_DOT",
    distance_strategy=DistanceStrategy.DOT_PRODUCT,
)

In [81]:
vector_store_dot.add_texts(list_chunks)

['728FA9C3B336D22E',
 '3F5B4C91A8B9CB3F',
 '36E07AAE0148C400',
 '1D9B921D2B2FF621',
 '9772A3E69BDE1011',
 '45E94D7187DAD0C0',
 '9A1D6EBC41A5A643',
 '8D783CBBF22CFBF7',
 '287A0FC500DFDF64',
 '584A5C35F516F11C',
 'F4A2845CA1D7765B',
 '444A9FEEE44558C9',
 '3E3BEDB401F195E0',
 '3A9F2FD33F8E56B1',
 'B7BA34A4217664BE',
 'A7DC596C455755FA',
 '583D822839ABFEB3',
 'F30E3BAD46F7D94F',
 'AF507CC44DD5B8A5',
 'EBC22181198D0820',
 'DCC0277E7C928E05',
 'A1860E32C30CF1CA',
 '040803B3D85D9908',
 '7B02F976FE07BDE5',
 'A6294751B6C34FF5',
 '7BC31156623F1A41',
 'F14C70392F4E4D7B',
 'D66712B917B1FF7F',
 '76500A7653419435',
 '0070B9E5E88488AA',
 'F4E05E3626C26B78',
 '8068B41E9680A457',
 '5E7E0146F332E558',
 'B0545DE5859B16C5',
 '51B05E8A99ED2FCD',
 'E76616E5B658B6AD',
 '5821EACDB7EDC912',
 '360A66D55A83C778',
 '5239AA01E6FF322F',
 'D5062F4C0F8B0794',
 'DEB86609470ED416',
 '69551EF94DA64CB4',
 '6B62ABC4F03978E0',
 '58B7AD05FBA90F68',
 '72608657223A7A2C',
 '871C4B8B8601A2C6',
 'A14031169F4E6393',
 'D9F39CADA8F

In [86]:
answers = vector_store_dot.similarity_search("diabetes in africa",  20)

In [87]:
for i in range(len(answers)):
    print(answers[i].page_content)

CONCLUSION: This study demonstrates that nonautoimmune diabetes occurs in underweight individuals in sub-Saharan Africa and is characterized by the absence of visceral adiposity, reduced late-phase insulin secretion, and greater insulin deficiency. These findings necessitate further studies to inform how the prevention, identification, and management of diabetes in such individuals can be individualized.
BACKGROUND: Type 2 diabetes is common in relatively lean individuals in sub-Saharan Africa. It is unclear whether phenotypic differences exist between underweight and normal-weight African patients with type 2 diabetes. This study compared specific characteristics between underweight (body mass index <18.5 kg/m) and normal-weight (body mass index of 18.5-24.9 kg/m) adult Ugandans with new-onset nonautoimmune diabetes.
[3·5-6·8]), the Improved Behavioural and Metabolic Risks scenario in north Africa and the Middle East (23·2% [20·2-26·5]), and the Improved Nutrition and Vaccination scen

### RAG Pipeline

In [88]:
template = """You are an assistant to the user, you are given some context below, please answer the query of the user with as detail as possible

Context:\"""
{context}
\"""

Question:\"
{question}
\"""

Answer:"""

In [109]:
from langchain_core.prompts import PromptTemplate

qa_prompt = PromptTemplate.from_template(template)

In [110]:
## User received Question
question = "Tell me something about Diabetest in africa"

In [130]:
docs = vector_store_dot.similarity_search(question, 5)
prompt = template.format(context=docs, question=question)

In [132]:
# Answer through RAG
oci_md.generate(prompts=[prompt])

**LLM Result**
**Text:**
[   Generation(text='\nAfrica has a high prevalence of diabetes, with a pooled prevalence of 8.3% among adults in sub-Saharan Africa, which is higher than the global average of 7.8%. The prevalence of diabetes varies widely across the region, with the highest rates found in north Africa and the Middle East (10.3%), followed by sub-Saharan Africa (8.3%), and the lowest rates found in south Africa (6.8%).\nThe study found that non-autoimmune diabetes is more common in underweight individuals in sub-Saharan Africa, characterized by the absence of visceral adiposity, reduced late-phase insulin secretion, and greater insulin deficiency. The study also found that the prevalence of poor sleep quality is high among diabetes patients in Ethiopia, with almost half of the patients having poor sleep quality.\nThe burden of diabetes in Africa is expected to increase in the future, with the largest shift towards morbidity and away from premature death forecasted for sub-Saha

In [133]:
# Direct Answer through LLM
oci_md.generate(prompts=[question])

**LLM Result**
**Text:**
[   Generation(text='\n\nDiabetes is a serious health problem in Africa, where it affects millions of people. According to the World Health Organization (WHO), Africa has the highest burden of diabetes in the world, with an estimated 18.3 million people living with the disease in 2019. This represents about 60% of the global total of people living with diabetes.\nThere are several factors that contribute to the high prevalence of diabetes in Africa, including:\n1. Urbanization and lifestyle changes: As people move from rural areas to cities, they often adopt unhealthy lifestyles, such as a diet high in sugar and fat, and a lack of physical activity. This can increase the risk of developing diabetes.\n2. Obesity: Many Africans are becoming increasingly overweight and obese, which is a major risk factor for developing diabetes.\n3. Lack of access to healthcare: In many parts of Africa, people do not have access to basic healthcare services, including diabetes dia